In [1]:
import dask
from dask.distributed import Client, progress
import dask.dataframe as dd
import numpy as np
import re
import dask_xgboost as dxgb
#from dask_ml.model_selection import train_test_split

In [2]:
dask.__version__

'2.5.2'

In [3]:
client = Client(n_workers=4, threads_per_worker=1)

In [4]:
client

Client Scheduler: tcp://127.0.0.1:52908 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.46 GB


In [5]:
train = dd.read_csv('train.csv')
train.compute()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [22]:
test = dd.read_csv('test.csv')
test.compute()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [7]:
gender = dd.read_csv('gender_submission.csv')
gender.compute()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [8]:
PassengerId = test['PassengerId']
full_data = [train, test]

# Some features of my own that I have added in
# Gives the length of the name
train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)
# Feature that tells whether a passenger had a cabin on the Titanic
train['Has_Cabin'] = train["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

# Feature engineering steps taken from Sina
# Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

# Remove all NULLS in the Embarked column
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    
# Remove all NULLS in the Fare column and create a new feature CategoricalFare
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(np.median(train['Fare']))

# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""
# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping titles
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)


C:\Users\8prab\Anaconda3\lib\site-packages\dask\dataframe\core.py:2957: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Name', 'int64'))

  warnings.warn(meta_warning(meta))
C:\Users\8prab\Anaconda3\lib\site-packages\dask\dataframe\core.py:2957: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Cabin', 'int64'))

  warnings.warn(m

In [9]:
def apply_fare(val):
    if val <= 7.91:
        return 0
    elif val > 7.91 and val <= 14.454:
        return 1
    elif val > 14.454 and val <= 31:
        return 2
    elif val > 31:
        return 3

In [10]:
def apply_age(val):
    if val <= 16:
        return 0
    elif val > 16 and val <= 32:
        return 1
    elif val > 32 and val <= 48:
        return 2
    elif val > 48 and val <= 64:
        return 3
    elif val > 64:
        return 4
    else:
        return 5

In [11]:
def apply_single(val):
    if val['FamilySize'] > 1:
        return 0
    else:
        return 1

In [12]:
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].apply(apply_fare)
    dataset['Age'] = dataset['Age'].apply(apply_age)
    dataset['Single'] = dataset.apply(lambda x : apply_single(x),axis=1)

C:\Users\8prab\Anaconda3\lib\site-packages\dask\dataframe\core.py:2957: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Fare', 'int64'))

  warnings.warn(meta_warning(meta))
C:\Users\8prab\Anaconda3\lib\site-packages\dask\dataframe\core.py:2957: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Age', 'int64'))

  warnings.warn(met

In [13]:
full_data[0].compute()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Name_length,Has_Cabin,FamilySize,Title,Single
0,1,0,3,"Braund, Mr. Owen Harris",1,1,1,0,A/5 21171,0,NaN,0,23,0,2,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,0,PC 17599,3,C85,1,51,1,2,3,0
2,3,1,3,"Heikkinen, Miss. Laina",0,1,0,0,STON/O2. 3101282,1,NaN,0,22,0,1,2,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,2,1,0,113803,3,C123,0,44,1,2,3,0
4,5,0,3,"Allen, Mr. William Henry",1,2,0,0,373450,1,NaN,0,24,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,1,0,0,211536,1,NaN,0,21,0,1,5,1
887,888,1,1,"Graham, Miss. Margaret Edith",0,1,0,0,112053,2,B42,0,28,1,1,2,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,5,1,2,W./C. 6607,2,NaN,0,40,0,4,2,0
889,890,1,1,"Behr, Mr. Karl Howell",1,1,0,0,111369,2,C148,1,21,1,1,1,1


In [14]:
# Feature selection
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp']
train = train.drop(drop_elements, axis = 1)
test  = test.drop(drop_elements, axis = 1)

In [15]:
train.compute()

,Survived,Pclass,Sex,Age,Parch,Fare,Embarked,Name_length,Has_Cabin,FamilySize,Title,Single
0,0,3,1,1,0,0,0,23,0,2,1,0
1,1,1,0,2,0,3,1,51,1,2,3,0
2,1,3,0,1,0,1,0,22,0,1,2,1
3,1,1,0,2,0,3,0,44,1,2,3,0
4,0,3,1,2,0,1,0,24,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,1,0,1,0,21,0,1,5,1
887,1,1,0,1,0,2,0,28,1,1,2,1
888,0,3,0,5,2,2,0,40,0,4,2,0
889,1,1,1,1,0,2,1,21,1,1,1,1


In [16]:
train, validation = train.random_split([0.80, 0.20])

In [21]:
train.compute()

1      1
3      1
4      0
6      0
8      1
      ..
881    0
883    0
886    0
887    1
890    0
Name: Survived, Length: 707, dtype: int64

In [17]:
params = {'objective': 'binary:logistic', 'nround': 1000,
          'max_depth': 16, 'eta': 0.01, 'subsample': 0.5,
          'min_child_weight': 1, 'tree_method': 'hist',
          'grow_policy': 'lossguide'}

bst = dxgb.train(client, params, train.drop('Survived', axis = 1), train['Survived'])

XGBoostError: [17:51:54] C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:60: Check failed: preds.Size() == info.labels_.Size() (1 vs. 707) : labels are not correctly providedpreds.size=1, label.size=707

In [35]:
bst

NameError: name 'bst' is not defined

In [36]:
predictions = dxgb.predict(client, bst, test).persist()

NameError: name 'bst' is not defined

Exception ignored in: <function RabitTracker.__del__ at 0x000002220A1624C8>
Traceback (most recent call last):
  File "C:\Users\8prab\Anaconda3\lib\site-packages\dask_xgboost\tracker.py", line 190, in __del__
    self.sock.close()
AttributeError: 'RabitTracker' object has no attribute 'sock'


In [64]:
predictions

dask.array<values, shape=(nan,), dtype=float32, chunksize=(nan,), chunktype=numpy.ndarray>

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
print(roc_auc_score(labels_test.compute(),predictions.compute()))